# Pipeline

- This notebook is used to test various aspects of the twitter pipeline 

## Imports

In [1]:
path = '../'
import pandas as pd
#import custom functions
from musow_pipeline.LogReg_Searches import LogRegSearches
from musow_pipeline import conf

## Variables

In [3]:
# descriptions training set -> v2 = musow+mji descriptions vs summarized scrapes from twitter searches  
archive_desc_training_v2 = pd.read_pickle(path+'LOGREG_RELEVANCE/TRAINING_SETS/archive_desc_training_v2.pkl')

# twitter training set -> v1 = tweets from bigrams vs tweets for digital humanities and music company 
twitter_training_set_v1 = pd.read_pickle(path+'LOGREG_RELEVANCE/TRAINING_SETS/twitter_training_v1.pkl')

In [2]:
# descriptions training set 
archive_desc_training_v4 = pd.read_pickle(path+'LOGREG_RELEVANCE/TRAINING_SETS/archive_desc_training_v4.pkl')

# twitter training set 
twitter_training_set_v2 = pd.read_pickle(path+'LOGREG_RELEVANCE/TRAINING_SETS/twitter_training_v2.pkl')
twitter_training_set_v2_alt = pd.read_pickle(path+'LOGREG_RELEVANCE/TRAINING_SETS/twitter_training_v2_alt.pkl')

## Training twitter and descriptions classifiers

This is a ONE TIME operation. The models are pickled and loaded later to predict new results

In [8]:
# one time training on twitter
twitter_training_model = LogRegSearches.train(twitter_training_set_v2, 'tweet', 'Target', 10, 'precision', 1000, 'twitter_v2', path)

# one time training on resources
#resource_training_model = LogRegSearches.train(archive_desc_training_v4, 'Description', 'Target', 10, 'precision', 1000, 'resources_v4',path)

report:
              precision    recall  f1-score   support

           0       0.70      0.72      0.71       556
           1       0.69      0.67      0.68       510

    accuracy                           0.70      1066
   macro avg       0.69      0.69      0.69      1066
weighted avg       0.69      0.70      0.69      1066



## Query Twitter

Calls Twitter API with the list of keywords and returns the table `prediction_twitter`

In [3]:
#load token
token = conf.TOKEN
#a selection of keywords based on MJI and musoW datasets

#Choose keywords  
keywords = ['music archive', 'music collection', 'audio file', 'music library', 'sheet music', 'sound archive', 'sound recording']

#search timeframe (if using custom search)
start = ['2022-05-22T00:00:00.000Z']
end = ['2022-05-28T23:59:59.000Z']

#choose search option 
## search last week
#tweets = LogRegSearches.search_twitter_weekly(token, keywords, 50, 50)
## search custom timeframe
tweets = LogRegSearches.search_twitter_custom(token, keywords, start, end, 500, 500)

-------------------
Token:  None
Endpoint Response Code: 200
-------------------
Start Date:  2022-05-22T00:00:00.000Z
# of Tweets added from this response:  49
Total # of Tweets added for '"music archive" -is:retweet': 49
-------------------
Total number of results: 49
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpywo625zwim847tptjf91gd1ryrul
Start Date:  2022-05-22T00:00:00.000Z
# of Tweets added from this response:  494
Total # of Tweets added for '"music collection" -is:retweet': 494
-------------------
-------------------
Token:  b26v89c19zqg8o3fpywo625zwim847tptjf91gd1ryrul
Endpoint Response Code: 200
-------------------
Start Date:  2022-05-22T00:00:00.000Z
# of Tweets added from this response:  160
Total # of Tweets added for '"music collection" -is:retweet': 654
-------------------
Total number of results: 654
-------------------
Token:  None
Endpoint Response Code: 200
-------------------
Start Date:  2022-05-22T00:00:00.000Z
# of 

## Classify tweets

In [2]:
#load all search results into a single dataframe 
tweets_to_classify = LogRegSearches.tweets_to_classify(path+'TWITTER_SEARCHES/RAW_SEARCHES/', '2022-05-22_05-28.pkl')
tweets_to_classify

Total tweets to classify: 801


,user,tweet id,created_at,lang,like_count,quote_count,reply_count,retweet_count,tweet,URL,Search KW
0,5555Hotaru,1530677324005314560,2022-05-28 22:28:07+00:00,en,0,0,0,0,Ryuichi Sakamoto &amp; Joe Hisaishi Best Piano...,https://youtu.be/k9vJKFfTu4A,"""music collection"" -is:retweet"
1,rockisinfinite,1530603012158201857,2022-05-28 17:32:49+00:00,en,1,0,0,0,Knuckles: Unknown From M.E. by Sonic’s Music C...,https://soundcloud.app.goo.gl/QEM6MTmDuGk6s1ps8,"""music collection"" -is:retweet"
2,MalafachaSka,1530602805559427078,2022-05-28 17:32:00+00:00,en,2,1,0,0,"Big Facts, started my music collection with ...",https://opensea.io/assets/matic/0xc29cbe04ae32...,"""music collection"" -is:retweet"
3,ro_p256,1530574324654292993,2022-05-28 15:38:50+00:00,en,0,0,0,0,Check out Pokemon Piano Sheet Music Collecti...,https://www.ebay.com/itm/225007047626?mkevt=1&...,"""music collection"" -is:retweet"
4,willpfeifer,1530565800897167360,2022-05-28 15:04:58+00:00,en,3,0,0,0,Don't forget to tune in TONIGHT at 9 PM ET / 8...,https://www.suburbspod.com/radio/,"""music collection"" -is:retweet"
...,...,...,...,...,...,...,...,...,...,...,...
796,DAJoseph16,1528323865021403137,2022-05-22 10:36:18+00:00,en,8,0,0,2,"Dear friends, Plz listen to this audio file. I...",http://www.dajoseph.com/Audio-Videos/Sunday_QA...,"""audio file"" -is:retweet"
797,MusicScheduling,1528290567431520259,2022-05-22 08:24:00+00:00,en,1,0,0,0,Unlike other music scheduling or playlist crea...,https://www.libraryplayer.co.uk/2021/07/musics...,"""audio file"" -is:retweet"
798,englishspeeches,1528226664118886401,2022-05-22 04:10:04+00:00,en,4,0,0,0,"Jackie Chan: “Now I am older, I understand we...","https://youtu.be/Y0B32z3WwwA, https://www.engl...","""audio file"" -is:retweet"
799,reconzar,1528221549429288960,2022-05-22 03:49:45+00:00,en,0,0,0,0,I also volunteer at a local museum/biosphere....,https://soundcloud.com/reconzar/hideo-kojima-s...,"""audio file"" -is:retweet"


In [16]:
#run classification and get links from results
predicted_tweets, twitter_link_list = LogRegSearches.predict_twitter(path, 'twitter_v2_alt', tweets_to_classify, 'tweet', 1)
predicted_tweets

Total tweets predicted: 504


,tweet,Prediction,Score,Probability,Input Length,URL,Search KW,tweet date,user,tweet id
0,Music Note Clock Wall Clock Music Teacher Gift...,1,11.547149,0.99999,169,https://ift.tt/3aEqwMr,"""sheet music"" -is:retweet",2022-05-24,pondscumsteph1,1529015312821125121
1,Fur Elise sheet music (Ludwig van Beethoven mu...,1,8.546705,0.999806,80,https://sheetmusic.kongashare.com/fur-elise-sh...,"""sheet music"" -is:retweet",2022-05-25,konga_share,1529256862230712320
2,"CINEMA PARADISO, ENNIO MORRICONE with sheet mu...",1,8.145584,0.99971,101,https://sheetmusiclibrary.website/2022/05/24/c...,"""sheet music"" -is:retweet",2022-05-24,LibrarySheet,1529177480149487616
3,Wagner - The Ride of the Valkyries (piano solo...,1,8.075519,0.999689,116,https://sheetmusiclibrary.website/2022/05/02/w...,"""sheet music"" -is:retweet",2022-05-27,LibrarySheet,1530261038968582145
4,Elton John – Rocket Man (Piano solo) with shee...,1,7.9908,0.999662,105,https://sheetmusiclibrary.website/2022/05/24/e...,"""sheet music"" -is:retweet",2022-05-24,LibrarySheet,1529176305358487552
...,...,...,...,...,...,...,...,...,...,...
499,Not a day goes by where I don't miss https://t...,1,0.199036,0.549595,84,http://What.CD,"""music collection"" -is:retweet",2022-05-28,stowball,1530436349651996672
500,Buy my sheetmusic 'If I Fell' on SheetMusicPlu...,1,0.178405,0.544483,72,http://www.sheetmusicplus.com/title/22160947,"""sheet music"" -is:retweet",2022-05-25,Clavi10Claudio,1529441719166676993
501,Buy my sheetmusic 'Summer Nights' on SheetMusi...,1,0.157613,0.539322,76,http://www.sheetmusicplus.com/title/20679578,"""sheet music"" -is:retweet",2022-05-26,Clavi10Claudio,1529920839596003347
502,Ready to say goodbye to streaming services? We...,1,0.077469,0.519358,92,http://p4k.in/z4uXGYp,"""music collection"" -is:retweet",2022-05-26,pitchfork,1529972974228496386


In [7]:
discard = ['youtu', '404', 'Not Found', 'bandcamp', 'ebay', 'It needs a human touch', 'Page not found', 'open.spotify.com', 'We\'re sorry...', 'Not Acceptable!', 'Access denied', '412 Error', 'goo.gl', 'instagr.am', 'soundcloud', 'apple.co', 'amzn', 'masterstillmusic', 'Facebook', 'facebook', 'sheetmusiclibrary.website', 'Unsupported browser', 'Last.fm', 'last.fm', 'amazon', 'tidal.com', 'tmblr.co', 'blogspot', 'dailymusicroll', 'PortalTaxiMusic', 'apple.news', 'yahoo.com', 'sheetmusicplus.com', 'musicnotes.com', 'musescore.com', 'etsy', 'nts.live', 'twitch.tv', 'YouTube', 'radiosparx.com', 'freemusicarchive.org', 'blastradio', 'opensea', 'mixcloud', 'catalog.works', 'nft', 'NFT', 'allmusic.com', 'foundation.app', 'Robot or human?', 'heardle', 'insession.agency', 'jobvite', 'career', 'docs.google.com/forms/', 'discogs.com', 'zora.co', 'play.google.com', 't.me', 'mintable.app', 'instagram', 'linkedin', 'forms.gle', 'vimeo', 'radioiita', 'spotify', 'event', 'mediafire', 'noodsradio', 'pinterest', 'rakuten', 'stackoverflow', 'fiverr', 'patreon']
tweets_to_classify = LogRegSearches.tweets_to_classify(path+'TWITTER_SEARCHES/RAW_SEARCHES/', '2022-05-22_05-28.pkl')
tweets_to_classify = tweets_to_classify.drop_duplicates(['tweet'], keep='last')
tweets_to_classify = tweets_to_classify[~tweets_to_classify.URL.str.contains('|'.join(discard))]
tweets_to_classify = tweets_to_classify[['tweet', 'URL', 'Search KW', 'created_at']]
tweets_to_classify['created_at'] = tweets_to_classify['created_at'].astype(str)
tweets_to_classify['created_at'] = tweets_to_classify['created_at'].str[0:10]
tweets_to_classify = tweets_to_classify.rename({'created_at': 'tweet date'}, axis=1)
twitter_link_list = [link for link in tweets_to_classify['URL']]

Total tweets to classify: 801


In [8]:
len(twitter_link_list)

395

## Scrape URLS

In [17]:
#scrape URL list and return a DF for resource classification
scraped_links = LogRegSearches.scrape_links(twitter_link_list, predicted_tweets, '2022-05-22_05-28_v3_alt')
scraped_links

1 https://ift.tt/3aEqwMr
2 https://sheetmusic.kongashare.com/fur-elise-sheet-music/
3 https://sheetmusiclibrary.website/2022/05/24/cinema-paradiso-morricone/
4 https://sheetmusiclibrary.website/2022/05/02/wagner-sheet-music-4/
5 https://sheetmusiclibrary.website/2022/05/24/elton-john-sheet-music-3/
6 https://sheetmusiclibrary.website/2022/05/23/beethoven-liszt-sheet-music/
7 https://dolarcalle.com/2022/05/25/vgm-ecce-valde-sheet-music-for-panzer-dragoon-vandal-game-music/
8 https://buff.ly/3wuiEXq
9 https://buff.ly/3LDHngk
10 https://sheetmusiclibrary.website/2022/05/27/jazz-play-along-4/


Your max_length is set to 120, but you input_length is only 103. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=51)


11 https://bit.ly/3MEwrAe
12 https://sheetmusiclibrary.website/2022/05/25/van-morrison-sheet-music/
13 https://sheetmusiclibrary.website/2022/05/22/universal-mind-of-bill-evans/


Token indices sequence length is longer than the specified maximum sequence length for this model (1386 > 1024). Running this sequence through the model will result in indexing errors


14 https://sheetmusiclibrary.website/2022/05/24/milestones-miles-davis/
15 https://sheetmusiclibrary.website/2022/05/03/jazz-play-along-3/?utm_source=ReviveOldPost&utm_medium=social&utm_campaign=ReviveOldPost
16 https://pin.it/3kvBUvm
17 https://sheetmusiclibrary.website/2022/02/15/spain-chick-corea-sheet-music/?utm_source=ReviveOldPost&utm_medium=social&utm_campaign=ReviveOldPost
18 https://sheetmusiclibrary.website/2022/04/01/scriabin-24-preludes-op-11/?utm_source=ReviveOldPost&utm_medium=social&utm_campaign=ReviveOldPost
19 https://sheetmusiclibrary.website/2022/05/24/milestones-miles-davis/?utm_source=ReviveOldPost&utm_medium=social&utm_campaign=ReviveOldPost
20 https://sunset-vending.com/product/chris-stapleton-smooth-as-tennessee-whiskey-sheet-music-single/
21 https://pin.it/6QYT8MZ


...

Your max_length is set to 120, but you input_length is only 88. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=44)


372 http://audio-file.org/2022/05/22/cleanfeed-online-recording-studio/
373 https://medium.com/@r.winfield/your-music-collection-is-so-fascinating-a-joy-ride-its-understandable-how-you-revisit-the-70-s-5ab6086c576d
374 http://www.sheetmusicplus.com/title/20663966
375 http://bit.ly/3GibcSq
376 http://www.sheetmusicplus.com/title/21708423
377 http://www.sheetmusicplus.com/title/22030540
378 http://www.sheetmusicplus.com/title/20768915
379 https://www.musicnotes.com/sheet-music/artist/wei-nin
380 https://www.sheetmusicplus.com/title/jesus-keep-me-near-the-cross-digital-sheet-music/22288674
381 https://www.sheetmusicplus.com/title/fy-gwanwyn-sky-digital-sheet-music/22286220
382 http://www.sheetmusicplus.com/title/20385871
383 https://assetstore.unity.com/packages/audio/music/orchestral/total-music-collection-89126?aid=1011l7qkc
384 https://buff.ly/3PUXIk2
385 https://qz.com/840661/what-cd-is-gone-a-eulogy-for-the-greatest-music-collection-in-the-world/
386 http://www.sheetmusicplus.com/tit

Your max_length is set to 120, but you input_length is only 114. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=57)


389 http://audio-file.org/2022/05/25/quiet-on-the-set-windows-volume-limiter/
337


,tweet,Prediction,Score,Probability,Input Length,URL,Search KW,tweet date,user,tweet id,Title,Description
0,Fur Elise sheet music (Ludwig van Beethoven mu...,1,8.546705,0.999806,80,https://sheetmusic.kongashare.com/fur-elise-sh...,"""sheet music"" -is:retweet",2022-05-25,konga_share,1529256862230712320,Fur Elise sheet music (Ludwig van Beethoven mu...,The music is composed of a well-known compose...
1,"CINEMA PARADISO, ENNIO MORRICONE with sheet mu...",1,8.145584,0.99971,101,https://sheetmusiclibrary.website/2022/05/24/c...,"""sheet music"" -is:retweet",2022-05-24,LibrarySheet,1529177480149487616,"CINEMA PARADISO, ENNIO MORRICONE With Sheet Mu...",The Creative Development of Johann Sebastian ...
2,Wagner - The Ride of the Valkyries (piano solo...,1,8.075519,0.999689,116,https://sheetmusiclibrary.website/2022/05/02/w...,"""sheet music"" -is:retweet",2022-05-27,LibrarySheet,1530261038968582145,Wagner - The Ride Of The Valkyries (piano Solo...,The Ride of the Valkyries or Valkyrie Ride is...
3,Elton John – Rocket Man (Piano solo) with shee...,1,7.9908,0.999662,105,https://sheetmusiclibrary.website/2022/05/24/e...,"""sheet music"" -is:retweet",2022-05-24,LibrarySheet,1529176305358487552,Elton John – Rocket Man (Piano Solo) With Shee...,Elton John: 'I want to write more musicals an...
4,"Beethoven-Liszt - Symphony No. 3, Op. 55 (Pian...",1,7.861649,0.999615,122,https://sheetmusiclibrary.website/2022/05/23/b...,"""music library"" -is:retweet",2022-05-23,LibrarySheet,1528689728219914241,"Beethoven-Liszt - Symphony No. 3, Op. 55 (Pian...","Beethoven-Liszt – Symphony No. 3, Op. 55 (Pi..."
...,...,...,...,...,...,...,...,...,...,...,...,...
332,This Month in Digital Music Libraries - May 20...,1,0.214171,0.553339,96,https://buff.ly/3PUXIk2,"""music library"" -is:retweet",2022-05-26,bliss_music,1529851364108554240,This month in digital music libraries - May 20...,This month in digital music libraries - May 2...
333,Buy my sheetmusic 'If I Fell' on SheetMusicPlu...,1,0.178405,0.544483,72,http://www.sheetmusicplus.com/title/22160947,"""sheet music"" -is:retweet",2022-05-25,Clavi10Claudio,1529441719166676993,If I Fell By John Lennon And Paul McCartney - ...,There are currently no items in your cart. W...
334,Buy my sheetmusic 'Summer Nights' on SheetMusi...,1,0.157613,0.539322,76,http://www.sheetmusicplus.com/title/20679578,"""sheet music"" -is:retweet",2022-05-26,Clavi10Claudio,1529920839596003347,Summer Nights By Warren Casey And Jim Jacobs -...,There are currently no items in your cart. W...
335,Ready to say goodbye to streaming services? We...,1,0.077469,0.519358,92,http://p4k.in/z4uXGYp,"""music collection"" -is:retweet",2022-05-26,pitchfork,1529972974228496386,Everything You Need to Build Your Own Digital ...,A Network Attached Storage (NAS) box is the i...


## Classify URLS

In [18]:
predicted_resources = LogRegSearches.predict_resource(path, 'resources_v4', scraped_links, 'Description', 1, '2022-05-22_05-28_v2+v4_alt')
predicted_resources

,tweet,Prediction,Score,Probability,Input Length,URL,Search KW,tweet date,user,tweet id,Title,Description
0,Anita Datta reviews In the deep – a collection...,1,5.546715,0.996115,137,https://www.musicteachermagazine.co.uk/review/...,"""sheet music"" -is:retweet",2022-05-25,MusicTeacherMag,1529451291356934145,Choral sheet music reviews: In the deep | Musi...,Choral sheet music reviews: In the deep – a c...
1,"Beethoven-Liszt - Symphony No. 3, Op. 55 (Pian...",1,3.685057,0.975519,255,https://sheetmusiclibrary.website/2022/05/23/b...,"""music library"" -is:retweet",2022-05-23,LibrarySheet,1528689728219914241,"Beethoven-Liszt - Symphony No. 3, Op. 55 (Pian...","Beethoven-Liszt – Symphony No. 3, Op. 55 (Pi..."
2,Is your child learning a musical instrument? D...,1,3.542830,0.971882,889,https://getkidsintomusic.com/2022/05/16/where-...,"""sheet music"" -is:retweet",2022-05-24,kids_into,1529110819094548481,Where to Buy Sheet Music Online?,There are good options for buying music onlin...
3,Hark! The Herald Angels Sing(Wind Quartet)\n\n...,1,3.526400,0.971430,138,https://is.gd/lM0COj,"""sheet music"" -is:retweet",2022-05-27,HarmonyTabs,1530132066167312386,Hark! The Herald Angels Sing (Wind Quartet) – ...,Hark! The Herald Angels Sing. is a jazzy win...
4,We've partnered with and to offer FREE unlim...,1,3.396769,0.967603,201,https://buff.ly/3LE8jfX,"""sheet music"" -is:retweet",2022-05-28,MakeMusicDayUK,1530473969903620097,Make Music Day UK 2022 | Sheet Music Downloads...,Download the official sheet music selections ...
...,...,...,...,...,...,...,...,...,...,...,...,...
70,Make your Disney parks music collection BIGGER...,1,0.139426,0.534800,195,https://drive.google.com/drive/folders/1qf-Gy5...,"""music collection"" -is:retweet",2022-05-27,somestuffisaid,1530191742770847745,Theme Park Soundtracks - Google Drive,The Magic Kingdom - Attractions & More - Back...
71,My sis and I loved this book growing up. I wo...,1,0.124273,0.531028,155,https://www.sheetmusicplus.com/title/the-best-...,"""sheet music"" -is:retweet",2022-05-28,motcorner,1530583893010702341,The Best Songs Ever - 6th Edition (Easy Piano)...,World's Largest Sheet Music Selection . Easy ...
72,Golliwogg’s Cakewalk - Download Sheet Music PD...,1,0.085796,0.521436,182,https://www.scoreexchange.com/scores/110038.html,"""sheet music"" -is:retweet",2022-05-24,hamarpercussion,1528974411046469632,Golliwogg’s Cakewalk - Download Sheet Music PD...,Four mallet vibraphone solo arrangement by Ha...
73,The Music Library of Choice https://t.co/nEIUk...,1,0.021356,0.505339,185,https://www.totim.audio/,"""music library"" -is:retweet",2022-05-27,MatthewBromham,1530102343756304384,TOTIM - Tracks from £2.50 | TOTIM,"With over 4000 tracks, TOTIM is the music lib..."


***

Compare final results for evaluation

In [36]:
baseline = pd.read_csv(path+'LOGREG_RELEVANCE/FINAL_RESULTS/Jan-April_2022_assessed_by_Marilena.csv')
baseline['source'] = 'baseline'
baseline_true = baseline.loc[baseline['Add to musoW'] != 'no']
baseline_false = baseline.loc[baseline['Add to musoW'] == 'no']

evaluate = pd.read_csv(path+'LOGREG_RELEVANCE/PREDICTIONS/all_keywords_2022-01-01_04-30_desc_v2_100maxfeats.csv')
evaluate['source'] = 'new'

In [37]:
baseline_true = baseline_true['URL'].to_dict()
baseline_true = dict([(value, key) for key, value in baseline_true.items()])
baseline_true.update((k,'true positive') for k in baseline_true)
baseline_false = baseline_false['URL'].to_dict()
baseline_false = dict([(value, key) for key, value in baseline_false.items()])
baseline_false.update((k,'false positive') for k in baseline_false)

In [38]:
evaluate['sort'] = evaluate['URL'].map(baseline_true)
evaluate.loc[evaluate['sort'] != 'true positive', 'sort'] = evaluate['URL'].map(baseline_false)
evaluate['sort'] = evaluate['sort'].fillna('new')

In [39]:
evaluate

,Unnamed: 0,tweet,Prediction,Score,Probability,Input Length,URL,Search KW,Title,Description,source,sort
0,0,https://t.co/OkEHI6XQu3 Music archive Wolfgang...,1,7.489973,0.999442,189,http://www.opusip.co.uk/2022/01/25/music-archi...,"""music archive"" -is:retweet",Music archive Wolfgang's Vault resolves copyri...,25th January 2022\nMusic archive Wolfgang’s Va...,new,false positive
1,1,Online concert archive Wolfgang's Vault has re...,1,6.473366,0.998458,308,https://reut.rs/3rTJWTx,"""music archive"" -is:retweet",Music archive Wolfgang's Vault resolves copyri...,Rocker Greg Kihn sued online archive over all...,new,false positive
2,2,Video Interview:\nFlorida Sound Archive With S...,1,5.960977,0.997429,113,https://www.musiceternal.com/News/2022/Florida...,"""sound archive"" -is:retweet",Florida Sound Archive With Sam Rosenthal,Interview with Sam Rosenthal of Projekt Record...,new,true positive
3,3,Inmates at HMP Perth engage with archive birds...,1,5.949058,0.997398,216,https://www.listentosteve.com/doingbird,"""oral history"" -is:retweet",Steve Urquhart // doing bird,Inmates at HMP Perth engage with archive bird...,new,true positive
4,4,With first-hand access to the original film ar...,1,5.705338,0.996683,285,https://buff.ly/38mYNQm,"""archive collection"" -is:retweet",Introducing… THE ARCHIVE COLLECTION – Regal Robot,The Archive Collection – by Regal Robot™Intro...,new,false positive
...,...,...,...,...,...,...,...,...,...,...,...,...
480,480,"Hard to believe there isn't, although it appe...",1,0.021968,0.505492,298,https://digital.library.unt.edu/ark:/67531/met...,"""digital library"" -is:retweet",Norma and Mel Gabler: The Development and Caus...,The problem of this study was to trace throug...,new,new
481,481,Join Delyna Baxter as she discusses the true h...,1,0.009815,0.502454,193,http://bit.ly/register-delyna22,"""oral history"" -is:retweet",Webinar Registration - Zoom,"Zoom is a full-featured, easy-to-use, engagin...",new,new
482,482,NowPlaying blues : Realize (Digital Edition Bo...,1,0.008761,0.502190,238,https://cowboysjukejoint.com/,"""digital edition"" -is:retweet",Cowboy's Juke Joint - Playing The Harder side ...,Cowboy’s Juke Joint is an independent non-com...,new,false positive
483,483,Concerts of the ensemble Bis-Quit band from St...,1,0.000688,0.500172,239,https://en.rwp.agency/news/953/,"""music culture"" -is:retweet",Concerts of the ensemble Bis-Quit band from St...,Kyrgyzstan will hold concerts of the Bis-Quit...,new,false positive


In [33]:
evaluate.to_csv(path+'LOGREG_RELEVANCE/FINAL_RESULTS/Jan-April_2022_desc_maxfeats_100.csv')

In [27]:
test = pd.read_pickle('/Users/laurentfintoni/Desktop/University/COURSE DOCS/THESIS/Internship/musow-pipeline-thesis/LOGREG_RELEVANCE/SCRAPES/2022-05-22_05-28_v3_alt.pkl')
test

,tweet,Prediction,Score,Probability,Input Length,URL,Search KW,tweet date,user,tweet id,Title,Description
0,Fur Elise sheet music (Ludwig van Beethoven mu...,1,8.546705,0.999806,80,https://sheetmusic.kongashare.com/fur-elise-sh...,"""sheet music"" -is:retweet",2022-05-25,konga_share,1529256862230712320,Fur Elise sheet music (Ludwig van Beethoven mu...,The music is composed of a well-known compose...
1,"CINEMA PARADISO, ENNIO MORRICONE with sheet mu...",1,8.145584,0.99971,101,https://sheetmusiclibrary.website/2022/05/24/c...,"""sheet music"" -is:retweet",2022-05-24,LibrarySheet,1529177480149487616,"CINEMA PARADISO, ENNIO MORRICONE With Sheet Mu...",The Creative Development of Johann Sebastian ...
2,Wagner - The Ride of the Valkyries (piano solo...,1,8.075519,0.999689,116,https://sheetmusiclibrary.website/2022/05/02/w...,"""sheet music"" -is:retweet",2022-05-27,LibrarySheet,1530261038968582145,Wagner - The Ride Of The Valkyries (piano Solo...,The Ride of the Valkyries or Valkyrie Ride is...
3,Elton John – Rocket Man (Piano solo) with shee...,1,7.9908,0.999662,105,https://sheetmusiclibrary.website/2022/05/24/e...,"""sheet music"" -is:retweet",2022-05-24,LibrarySheet,1529176305358487552,Elton John – Rocket Man (Piano Solo) With Shee...,Elton John: 'I want to write more musicals an...
4,"Beethoven-Liszt - Symphony No. 3, Op. 55 (Pian...",1,7.861649,0.999615,122,https://sheetmusiclibrary.website/2022/05/23/b...,"""music library"" -is:retweet",2022-05-23,LibrarySheet,1528689728219914241,"Beethoven-Liszt - Symphony No. 3, Op. 55 (Pian...","Beethoven-Liszt – Symphony No. 3, Op. 55 (Pi..."
...,...,...,...,...,...,...,...,...,...,...,...,...
332,This Month in Digital Music Libraries - May 20...,1,0.214171,0.553339,96,https://buff.ly/3PUXIk2,"""music library"" -is:retweet",2022-05-26,bliss_music,1529851364108554240,This month in digital music libraries - May 20...,This month in digital music libraries - May 2...
333,Buy my sheetmusic 'If I Fell' on SheetMusicPlu...,1,0.178405,0.544483,72,http://www.sheetmusicplus.com/title/22160947,"""sheet music"" -is:retweet",2022-05-25,Clavi10Claudio,1529441719166676993,If I Fell By John Lennon And Paul McCartney - ...,There are currently no items in your cart. W...
334,Buy my sheetmusic 'Summer Nights' on SheetMusi...,1,0.157613,0.539322,76,http://www.sheetmusicplus.com/title/20679578,"""sheet music"" -is:retweet",2022-05-26,Clavi10Claudio,1529920839596003347,Summer Nights By Warren Casey And Jim Jacobs -...,There are currently no items in your cart. W...
335,Ready to say goodbye to streaming services? We...,1,0.077469,0.519358,92,http://p4k.in/z4uXGYp,"""music collection"" -is:retweet",2022-05-26,pitchfork,1529972974228496386,Everything You Need to Build Your Own Digital ...,A Network Attached Storage (NAS) box is the i...


In [1]:
from langdetect import detect

detect(' youtube爵士教学-bebop. youtube牵士. youtube===Barry Harris: Bebop, bebop . Youtube==Bebop-style: Beop, jazz, funk, jazz piano, funk .')

'en'

In [22]:
def detect_lang(text):
    return detect(text)

test['Description'].apply(detect_lang)

LangDetectException: No features in text.

In [55]:
import difflib

archive_desc_training = pd.read_pickle(path+'LOGREG_RELEVANCE/TRAINING_SETS/archive_desc_training_v4.pkl')
test = pd.read_csv('/Users/laurentfintoni/Desktop/University/COURSE DOCS/THESIS/Internship/musow-pipeline-thesis/LOGREG_RELEVANCE/PREDICTIONS/May 2022 tests/best_keywords_2022-05-08_-05-14_v2+v4_alt.csv')

In [31]:
checklist = archive_desc_training.loc[archive_desc_training['Target'] == 1]
checklist = checklist['URL'].to_list()

In [32]:
from urllib.parse import urlparse
parsed = []
for url in checklist:
    parsedurl = urlparse(url)
    parsed.append(parsedurl[1])


In [33]:
parsed = set(parsed)

In [25]:
import re
test['Match'] = test['URL'].apply(lambda x: re.search(rf'({x})', '|'.join(parsed)))


In [58]:
def match_url(row):
    if row['URL'] in '|'.join(parsed):
        return 'training set match'
    else:
        pass

In [59]:
test['Match'] = test.apply(match_url, axis=1)
test['Match'] = test['Match'].astype(str)

In [60]:
test.loc[test['Match'] != 'None']

,Unnamed: 0,user,created_at,lang,like_count,quote_count,reply_count,retweet_count,tweet,URL,Search KW,Title,Description,Prediction,Score,Probability,Input Length,Match


In [63]:
test.loc[test['URL'].str.contains('|'.join(parsed), case=False, regex=True), ['Match']] = 'training set match'

In [64]:
test.loc[test['Match'] == 'training set match']

,Unnamed: 0,user,created_at,lang,like_count,quote_count,reply_count,retweet_count,tweet,URL,Search KW,Title,Description,Prediction,Score,Probability,Input Length,Match
0,0,tom2tec,2022-05-08 09:56:30+00:00,en,0,0,0,0,Music Player GO ~ Android Audio Player https:/...,http://audio-file.org/2022/05/08/music-player-...,"""audio file"" -is:retweet",Music Player GO ~ Android Audio Player | The A...,"Home of Music Player GO, a minimal yet fully-...",1,4.566412,0.989712,216,training set match
3,3,matonizz,2022-05-11 22:03:54+00:00,en,3,0,0,0,Such a rich sound... Hard not to love the bass...,https://www.matonizz.com/product/didziausia-me...,"""sheet music"" -is:retweet",DIDŽIAUSIA MEILĖ (A Love Most Supreme) - David...,The $14.99 price tag is $1.99 for sheet music...,1,3.291026,0.964120,116,training set match
4,4,soundarchive,2022-05-12 14:57:22+00:00,en,14,0,1,2,This is a great listen. from the sound archiv...,https://www.bbc.co.uk/sounds/play/m001692r,"""sound archive"" -is:retweet",Between the Ears - A Society of Recordists - B...,A sonic portrait of the Wildlife Sound Record...,1,3.220357,0.961593,167,training set match
5,5,matonizz,2022-05-13 20:21:47+00:00,en,3,0,0,2,"Super Pleasant Music, Debussy's ""The Girl With...",https://www.matonizz.com/product/the-girl-with...,"""sheet music"" -is:retweet",THE GIRL WITH THE FLAXEN HAIR - Claude Debussy...,The $14.99 sheet music includes five backing ...,1,2.904269,0.948057,151,training set match
10,10,jingajik,2022-05-13 03:29:14+00:00,en,11,1,2,5,"John Cousins, sonic artist - Part 2 of the blo...",https://natlib.govt.nz/blog/posts/tales-from-t...,"""music collection"" -is:retweet",Tales from the Acousmonium – Part 2 | National...,The John Cousins Collection contains an exten...,1,1.449516,0.809924,898,training set match
27,27,phillipsme,2022-05-08 07:03:29+00:00,en,0,0,1,0,National Sound Archive?\nhttps://t.co/Gmejkx...,https://sounds.bl.uk/Accents-and-dialects/Surv...,"""sound archive"" -is:retweet",Survey of English Dialects - Accents and diale...,Survey of English Dialects (SED) was a ground...,1,0.312134,0.577406,547,training set match
32,32,practicingmusic,2022-05-12 16:23:11+00:00,en,0,0,0,0,Practicing Musician offers K-12 music educator...,https://www.practicingmusician.com,"""sheet music"" -is:retweet",Practicing Musician,Online Concert Band and Strings Methods. For ...,1,0.099019,0.524735,175,training set match
